In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, confusion_matrix, classification_report
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.neighbors import LocalOutlierFactor


## Carregamento dos Dados

In [ ]:

data = pd.read_csv('train.csv')
data.head()


## Análise Exploratória dos Dados (EDA)

In [ ]:

print(data.info())
print(data.describe())
print(data.isnull().sum().sort_values(ascending=False).head(10))

plt.figure(figsize=(12,8))
sns.heatmap(data.corr(numeric_only=True), cmap='coolwarm', annot=False)
plt.title('Mapa de Calor das Correlações')
plt.show()

sns.histplot(data['SalePrice'], kde=True)
plt.title('Distribuição dos Preços de Venda')
plt.show()


## Feature Engineering

In [ ]:

corr = data.corr(numeric_only=True)['SalePrice'].abs().sort_values(ascending=False)
top_features = corr[1:11].index.tolist()

df = data[top_features + ['SalePrice']].copy()
df = df.dropna()

scaler = StandardScaler()
X = scaler.fit_transform(df.drop('SalePrice', axis=1))
y = df['SalePrice']


## Modelagem Supervisionada - Regressão Linear

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)
print("R2:", r2_score(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))


## Modelagem Supervisionada - Classificação

In [ ]:

threshold = y.median()
y_class = ['Alto' if price >= threshold else 'Baixo' for price in y]

X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X, y_class, test_size=0.2, random_state=42)

clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_c, y_train_c)

y_pred_c = clf.predict(X_test_c)
print("Acurácia:", accuracy_score(y_test_c, y_pred_c))
print(confusion_matrix(y_test_c, y_pred_c))
print(classification_report(y_test_c, y_pred_c))


## Modelagem Não Supervisionada - Clusterização

In [ ]:

kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(X)

plt.scatter(X[:, 0], X[:, 1], c=clusters, cmap='viridis')
plt.title('Clusterização das Casas')
plt.xlabel(top_features[0])
plt.ylabel(top_features[1])
plt.show()


## Redução de Dimensionalidade

In [ ]:

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

plt.scatter(X_pca[:,0], X_pca[:,1], c=clusters, cmap='rainbow')
plt.title('PCA - Redução de Dimensionalidade')
plt.show()


## Análise de Outliers

In [ ]:

lof = LocalOutlierFactor(n_neighbors=20)
outlier_flags = lof.fit_predict(X)

plt.scatter(X_pca[:,0], X_pca[:,1], c=outlier_flags, cmap='coolwarm')
plt.title('Outlier Detection com LOF')
plt.show()


## Conclusão

In [ ]:

# O trabalho mostrou que é possível prever os preços com Regressão Linear (R2 e RMSE)
# e classificar preços como 'Alto' ou 'Baixo' usando Random Forest com boa acurácia.
# Além disso, a clusterização revelou grupos semelhantes e a análise de outliers identificou pontos fora do padrão.
